In [47]:
from __future__ import division

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from collections import deque


In [2]:
input_image = plt.imread('train_pic/10003_left.jpeg')
plt.imshow(input_image)
plt.show()

In [48]:
def flood_fill(input_image, threshold):
    segment_counter = 0
    segment_res = np.zeros((input_image.shape[0], input_image.shape[1]))
    color_mean = []
    #debug
    print segment_res.shape
    covered_pixel = 0
    #debug
    for i in range(input_image.shape[0]):
        for j in range(input_image.shape[1]):
            if (segment_res[i][j] == 0):
                queue_list = deque([(i, j)])
                segment_counter = segment_counter + 1
                segment_res[i][j] = segment_counter
                regeion_color0 = [input_image[i][j][0]]
                regeion_color1 = [input_image[i][j][1]]
                regeion_color2 = [input_image[i][j][2]]
                #
                print 'start covering', covered_pixel
                covered_pixel = covered_pixel + 1
                #
                while (len(queue_list) > 0):
                    center_i = queue_list[0][0]
                    center_j = queue_list[0][1]
                    for new_i in range(center_i - 1, center_i + 2):
                        for new_j in range(center_j - 1, center_j + 2):
                            if (new_i >= 0 and new_i < input_image.shape[0] and new_j >= 0 and new_j < input_image.shape[1]):
                                delta0 = input_image[center_i][center_j][0] - input_image[new_i][new_j][0]
                                delta1 = input_image[center_i][center_j][1] - input_image[new_i][new_j][1]
                                delta2 = input_image[center_i][center_j][2] - input_image[new_i][new_j][2]
                                color_distance = (delta0 ** 2) + (delta1 ** 2) + (delta2 ** 2)
                                if (segment_res[new_i][new_j] == 0 and color_distance <= threshold):
                                    queue_list.append((new_i, new_j))
                                    segment_res[new_i][new_j] = segment_counter
                                    regeion_color0.append(input_image[new_i][new_j][0])
                                    regeion_color1.append(input_image[new_i][new_j][1])
                                    regeion_color2.append(input_image[new_i][new_j][2])
                                    #
                                    covered_pixel = covered_pixel + 1
                                    #
                    queue_list.popleft()
                    color_mean.append((pd.Series(regeion_color0).median(), pd.Series(regeion_color1).median(), pd.Series(regeion_color2).median()))
                    if (covered_pixel % 100 == 0):
                        print 'has covered', covered_pixel
                # debug
                print 'covered', covered_pixel
                #
    res_pic = np.zeros(input_image.shape, dtype = np.uint8)
    for i in range(res_pic.shape[0]):
        for j in range(res_pic.shape[1]):
            res_pic[i][j][0] = color_mean[segment_res[i][j] - 1][0]
            res_pic[i][j][1] = color_mean[segment_res[i][j] - 1][1]
            res_pic[i][j][2] = color_mean[segment_res[i][j] - 1][2]
    return segment_res, res_pic

In [49]:
input_image.shape

(3168, 4752, 3)

In [50]:
segment_res, res_pic = flood_fill(input_image, 0.1)
plt.imshow(res_pic)
plt.show()

(3168, 4752)
start covering 0
has covered 100
has covered 200
has covered 300
has covered 400
has covered 500
has covered 600
has covered 700
has covered 800
has covered 900
has covered 1000
has covered 1100
has covered 1200
has covered 1300
has covered 1400
has covered 1500
has covered 1600
has covered 1700
has covered 1800
has covered 1900
has covered 2000
has covered 2100
has covered 2200
has covered 2300
has covered 2400
has covered 2500
has covered 2600
has covered 2700
has covered 2800
has covered 2900
has covered 3000
has covered 3100
has covered 3200
has covered 3300
has covered 3400
has covered 3500
has covered 3600
has covered 3700
has covered 3800
has covered 3900
has covered 4000
has covered 4100
has covered 4200
has covered 4300
has covered 4400
has covered 4500
has covered 4600
has covered 4700
has covered 4800
has covered 4900
has covered 5000
has covered 5100
has covered 5200
has covered 5300
has covered 5400
has covered 5500
has covered 5600
has covered 5700
has covere

KeyboardInterrupt: 

In [40]:
test_list = [(10, 8), (9, 2), (23, 4)]
print test_list[1][1]

2


In [22]:
input_image[1000][2000]

array([202, 174, 134], dtype=uint8)